In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

# For LangSmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [11]:
# created langchain environment in conda and selected that as kernel to run the code
from langchain_openai import ChatOpenAI

# no need to pass api key as we already loaded it from .env file
llm=ChatOpenAI(model='gpt-3.5-turbo')
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x00000223097C8A90> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000223097E05B0> root_client=<openai.OpenAI object at 0x000002230971C5E0> root_async_client=<openai.AsyncOpenAI object at 0x00000223097C8CD0> model_kwargs={} openai_api_key=SecretStr('**********')


In [ ]:
# passing input and getting response from the llm
result=llm.invoke("explain gen ai")
print(result)

#### PromptTemplate
PromptTemplate is something using which we can instruct our llm model how to behave. PromptTemplate take as input a dictionary, where each key represents a variable in the prompt template to fill in. 
- ChatPromptTemplate

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages([
    ("system","You are an expert AI Engineer. Provide me answer based on the question"),
    ("user","{input}")
])
print(prompt)

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an expert AI Engineer. Provide me answer based on the question'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


#### Reterieval and document chain: Langchain Expression Language (LCEL)
- Chaining using | operator: this | is saying that we need to combine all things connected with | and use it in passing to llm
- create_stuff_doccuments_chain(pass whatever you what to combine): it is a runnable binding

In [ ]:
chain=prompt|llm
response=chain.invoke({"input":"can you tell me about langsmith?"})
print(response)

#### Output Parser
It is responsible for providing the response how we want to display it. 
We can also create our own custom output parsers.
- StrOutputParser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

chain=prompt|llm|output_parser
response=chain.invoke({"input":"can you tell me about langsmith?"})
print(response)

#### Simple GEN AI App using OpenAI
We are going to scrape one web page and then ask answer based on that using llm

Steps:
- load .env file api keys
- load or scrap the data from web page - DataLoaders
- create chunk of loaded document - Splitting
- using embedding convert the chunk in vector stores
- querying the vector store using similarity search
- create llm , prompt and use retrieval chain - important 
- reterivers using create_retrieval_chain()
- get the response by invoking the retriever_chain created


In [25]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")
web_content=loader.load()
print(web_content)

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='\n\n\n\n\nBuild a Simple LLM Application | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a Simple LLM ApplicationBuild a Query Analysis SystemBuild a ChatbotConversational RAGBuild an Ex

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

web_split=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
web_chunks=web_split.split_documents(web_content)
print(web_chunks)

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a Simple LLM Application | 🦜️🔗 LangChain'), Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM applicati

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

openai_embeddings=OpenAIEmbeddings()
faiss_db=FAISS.from_documents(web_chunks,openai_embeddings)

In [ ]:
query="question related to loaded documents"
result=faiss_db.similarity_search(query)
print(result[0].page_content)

In [ ]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI()

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the question based on provided context.
<context>
{context}
</context>
"""
)

document_chain=create_stuff_documents_chain(llm, prompt)
print(document_chain)

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"your question here",
    "context": [Document(page_content="context you want to provide")]
})

#### if using reterievers then after creating vectorstores convert that in retrievers

In [ ]:
retriever=faiss_db.as_retriever()

In [ ]:
from langchain.chains import create_retrieval_chain

# using this, the context automatically gets in prompt template
retriever_chain=create_retrieval_chain(retriever,document_chain)

In [ ]:
final_response=retriever_chain.invoke({"input":"question here"})
print(final_response)

### Simple Gen AI App using Ollama

In [29]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

ollama_embedding=OllamaEmbeddings(model='llama3')
ollama_faiss_db=FAISS.from_documents(web_chunks,ollama_embedding)

C:\Users\srish\AppData\Local\Temp\ipykernel_27456\2621887515.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embedding=OllamaEmbeddings(model='llama3')


In [36]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate

ollama_llm=Ollama(model="llama3")
ollama_prompt=ChatPromptTemplate.from_template(
    """ 
Answer the question based on the provided context.
<context>
{context}
</context>
"""
)

In [37]:
from langchain.chains.combine_documents import create_stuff_documents_chain

ollama_doc_chain=create_stuff_documents_chain(ollama_llm,ollama_prompt)

In [38]:
ollama_retriever=ollama_faiss_db.as_retriever()

In [41]:
from langchain.chains import create_retrieval_chain

ollama_retriever_chain=create_retrieval_chain(ollama_retriever,ollama_doc_chain)
ollama_response=ollama_retriever_chain.invoke({"input":"jupyter setup"})
print(ollama_response)
print(ollama_response['answer'])

{'input': 'jupyter setup', 'context': [Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Simple LLM Application | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='ConversationBufferWindowMemory or ConversationTokenBufferMemoryMigrating off ConversationSummaryMemory or ConversationSummaryBufferMemoryA Long-Term Memory AgentRelease policySecurityTutorialsBuild a Simple LLM ApplicationOn this pageBuild a Simple LLM Application'), Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a Sim

### Conversational RAG
Enable a chatbot experience over an external source of data.

Agents: Build a chatbot that can take actions

In [42]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [43]:
from langchain_groq import ChatGroq

groq_llm=ChatGroq(model="Gemma2-9b-It")

In [45]:
from langchain_core.messages import HumanMessage

groq_llm.invoke([HumanMessage(content="Hi My name is Srishti and I am Full stack ai engineer")])

AIMessage(content="Hello Srishti, \n\nIt's nice to meet you! That's a really impressive title - Full Stack AI Engineer.  \n\nWhat kind of projects are you working on these days? \n\nI'm always eager to learn more about what people are doing in the field of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 23, 'total_tokens': 88, 'completion_time': 0.118181818, 'prompt_time': 0.000139819, 'queue_time': 0.01447786, 'total_time': 0.118321637}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-690ebdc7-dc97-437d-a5ba-708569320e51-0', usage_metadata={'input_tokens': 23, 'output_tokens': 65, 'total_tokens': 88})

whatever we are providing in the list of messages, llm is going to remember

In [46]:
from langchain_core.messages import AIMessage

groq_llm.invoke([
    HumanMessage(content="Hi My name is Srishti and I am Full stack ai engineer"),
    AIMessage(content="Hello Srishti, \n\nIt's nice to meet you! That's a really impressive title - Full Stack AI Engineer.  \n\nWhat kind of projects are you working on these days? \n\nI'm always eager to learn more about what people are doing in the field of AI."),
    HumanMessage(content="Hey what's my name and what I do?")
])

AIMessage(content="You are Srishti, and you are a Full Stack AI Engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  I'm happy to chat. \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 106, 'total_tokens': 153, 'completion_time': 0.085454545, 'prompt_time': 0.003663166, 'queue_time': 0.010866852, 'total_time': 0.089117711}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-16b11ddb-e581-493f-8cf7-98d1170e983b-0', usage_metadata={'input_tokens': 106, 'output_tokens': 47, 'total_tokens': 153})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

**get_session_history():** whenever we pass a session_id, it should be able to check whether the session_id is present in store dictionary or not. If it is present, we are going to get the entire chat message history for that session_id and will return it. It will give us response type of *BaseChatMessagehistory-(abstract base class for storing chat message history)* and *ChatMessageHistory-(in memory implementation of chat message history. stores messages in memory list).*

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

# to interaction with llm based on chat history
with_message_history=RunnableWithMessageHistory(groq_llm,get_session_history)

In [51]:
config={"configurable":{"session_id":"chat1"}}

In [52]:
with_message_history.invoke([
    HumanMessage(content="hi, my name is srishti and i am a full stack ai engineer")
], config=config)

AIMessage(content="Hi Srishti, it's nice to meet you!\n\nThat's a really impressive title - a full stack AI engineer! \n\nWhat kind of projects are you currently working on?  I'm always curious to learn about the innovative things people are doing with AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 25, 'total_tokens': 86, 'completion_time': 0.110909091, 'prompt_time': 0.000151659, 'queue_time': 0.014284738, 'total_time': 0.11106075}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b24ee7d1-da0d-40f6-96ff-947badb3fe5e-0', usage_metadata={'input_tokens': 25, 'output_tokens': 61, 'total_tokens': 86})

In [53]:
with_message_history.invoke([
    HumanMessage(content="what is my name and what i do?")
], config=config)

AIMessage(content="You told me your name is Srishti and that you are a full stack AI engineer!  \n\nIs there anything else you'd like to tell me about yourself or your work? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 103, 'total_tokens': 146, 'completion_time': 0.078181818, 'prompt_time': 0.003680355, 'queue_time': 0.010796995, 'total_time': 0.081862173}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4f4960ce-a07e-4037-936c-a784053a832f-0', usage_metadata={'input_tokens': 103, 'output_tokens': 43, 'total_tokens': 146})

In [56]:
# changing the session_id
config1={"configurable":{"session_id":"chat2"}}

In [57]:
with_message_history.invoke([
    HumanMessage(content="what is my name and what i do?")
], config=config1)

AIMessage(content="Since I don't have access to any information about you, including your name or profession, I can't tell you what they are.  \n\nIs there anything else I can help you with? Perhaps you'd like to play a game or have me write a story? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 91, 'total_tokens': 153, 'completion_time': 0.112727273, 'prompt_time': 0.002590809, 'queue_time': 0.011916938, 'total_time': 0.115318082}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c07653e9-5668-425e-a3f9-f6e7fb727f71-0', usage_metadata={'input_tokens': 91, 'output_tokens': 62, 'total_tokens': 153})

Using PromptTemplate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the question to best of your ability"),
        MessagesPlaceholder(variable_name="user_input")
    ]
)

chain=prompt|groq_llm

chain.invoke({"user_input":[HumanMessage(content="Hey my name is srishti agrawal")]})


AIMessage(content="Hello Srishti Agrawal! 👋 \n\nIt's nice to meet you.  \n\nWhat can I do for you today? I'm happy to answer questions, provide information, or even have a chat. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 34, 'total_tokens': 84, 'completion_time': 0.090909091, 'prompt_time': 0.001178506, 'queue_time': 0.09263901000000001, 'total_time': 0.092087597}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-608b58f0-5b25-48f7-b46c-d6297ba5bda7-0', usage_metadata={'input_tokens': 34, 'output_tokens': 50, 'total_tokens': 84})

In [60]:
config2={"configurable":{"session_id":"chat3"}}
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

with_message_history.invoke(
    [HumanMessage(content="Hey my name is srishti agrawal")],
    config=config2
)

AIMessage(content="Hello Srishti Agrawal, it's nice to meet you! \n\nWhat can I do for you today? I'm ready to answer your questions and help in any way I can. 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 34, 'total_tokens': 79, 'completion_time': 0.081818182, 'prompt_time': 0.000460778, 'queue_time': 0.012687692, 'total_time': 0.08227896}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-122bf3a7-18d5-44e8-9bcc-757eeb9ad569-0', usage_metadata={'input_tokens': 34, 'output_tokens': 45, 'total_tokens': 79})

PromptTemplate with multiple input variables

In [61]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","Tou are a helpful assistant. Answer all the question to the best of your ability in {lang}"),
        MessagesPlaceholder(variable_name="user_input")
    ]
)

chain=prompt|groq_llm

In [63]:
chain.invoke({"user_input":[HumanMessage(content="hey my name is srishti agrawal")],"lang":"hindi"})

AIMessage(content='Namaste Srishti Agrawal!  😊 \n\nAap kitni madad chahti hain? Mujhe kuch bhi pooch lijiye, main apke samay mein jawab dene ki koshish karungi. \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 37, 'total_tokens': 88, 'completion_time': 0.092727273, 'prompt_time': 0.000433868, 'queue_time': 0.014062469999999999, 'total_time': 0.093161141}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c3989c8c-ff3a-4ada-8e76-a862259c2909-0', usage_metadata={'input_tokens': 37, 'output_tokens': 51, 'total_tokens': 88})

Let's now wrap this more complicated chain in a message history class. this time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [64]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="user_input"
)

In [67]:
config3={"configurable":{"session_id":"chat4"}}
with_message_history.invoke(
    {
        "user_input":[HumanMessage(content="Hey, im srishti")],
        "lang":"hindi"
    },
    config=config3
)

AIMessage(content='नमस्ते, श्रीष्टि! 😊 \n\nअब मुझे आपका नाम पता चल गया।  \n\nक्या मैं आपकी कोई मदद कर सकता हूँ? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 110, 'total_tokens': 149, 'completion_time': 0.070909091, 'prompt_time': 0.003348287, 'queue_time': 0.010145149, 'total_time': 0.074257378}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-c36a3cd0-3441-4700-a4cd-daf0fe95703f-0', usage_metadata={'input_tokens': 110, 'output_tokens': 39, 'total_tokens': 149})

In [68]:
with_message_history.invoke(
    {
        "user_input":[HumanMessage(content="what's my name?")],
        "lang":"hindi"
    },
    config=config3
)

AIMessage(content='आपका नाम श्रीष्टि है। 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 163, 'total_tokens': 177, 'completion_time': 0.025454545, 'prompt_time': 0.005061642, 'queue_time': 0.008036958, 'total_time': 0.030516187}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f96c71b4-c360-48b0-9750-137c89d22fcb-0', usage_metadata={'input_tokens': 163, 'output_tokens': 14, 'total_tokens': 177})

### Managing the conversation history
One important concept to understand when buildinng chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. therfore, it is important to add a step that limits the size of the messages you are passing in.

**trim_messages**- helper to reduce how many messages we're sending to the model. the trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [102]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer=trim_messages(
    max_tokens=35,
    strategy="last",
    token_counter=groq_llm,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

conversation_chat=[
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi i am bob"),
    AIMessage(content="hi"),
    HumanMessage(content="i like ice cream"),
    AIMessage(content="which flavour"),
    HumanMessage(content="chocolate"),
    AIMessage(content="nice"),
    HumanMessage(content="remember 2+2 is 5"),
    AIMessage(content="ok sure"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yup"),
]

trimmer.invoke(conversation_chat)

[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='remember 2+2 is 5', additional_kwargs={}, response_metadata={}),
 AIMessage(content='ok sure', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yup', additional_kwargs={}, response_metadata={})]

In [107]:
# by using the itemgetter, we will be able to retrive the entire chat whatever is there in the prompttemplate
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(conversation_chat=itemgetter("user_input")|trimmer)
    | prompt
    | groq_llm
)

chain.invoke(
    {
        "user_input":conversation_chat + [HumanMessage(content="what ice cream i like")],
        "lang":"english"
    }
)

AIMessage(content='You said you like chocolate ice cream! 😊 🍫  \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 128, 'total_tokens': 154, 'completion_time': 0.047272727, 'prompt_time': 0.003676978, 'queue_time': 0.010583491, 'total_time': 0.050949705}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-64cbedf6-735a-4965-8643-60a6babe3445-0', usage_metadata={'input_tokens': 128, 'output_tokens': 26, 'total_tokens': 154})

In [108]:
# let's wrap this in message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="user_input"
)

config4={"configurable":{"session_id":"chat5"}}

with_message_history.invoke(
    {
        "user_input":conversation_chat + [HumanMessage(content="what's maths question i asked")],
        "lang":"english"
    },
    config=config4
)

AIMessage(content='You asked "What is 2+2?". \n\nI answered 4.  \n\nLet me know if you have any other math questions! \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 682, 'total_tokens': 716, 'completion_time': 0.061818182, 'prompt_time': 0.025010125, 'queue_time': 0.007981313999999996, 'total_time': 0.086828307}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6c509c22-d41d-40b5-a624-e0d9fcf6b84b-0', usage_metadata={'input_tokens': 682, 'output_tokens': 34, 'total_tokens': 716})